<a href="https://colab.research.google.com/github/data-maven-chetan/data-engineering/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=60f1adc55a5998630d1df2c99e4b9b7a96b168cbf791951baf98ac680fb39424
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql.functions import col

In [20]:
spark = SparkSession.builder.appName("Employee").getOrCreate()

In [94]:
data = [(1,'chetan',5000,'M'),(2,'Rohan',7800,'M'),\
        (3,'Rohit',9800,'M'),(4,'Rohini',7800,'F')\
        ,(5,'Roshita',7400,'F'),(6,'Shalini',5800,'F')]

header = ["id","name","salary","gender"]

In [95]:
emp_df= spark.createDataFrame(data,header)

In [96]:
emp_df.show()

+---+-------+------+------+
| id|   name|salary|gender|
+---+-------+------+------+
|  1| chetan|  5000|     M|
|  2|  Rohan|  7800|     M|
|  3|  Rohit|  9800|     M|
|  4| Rohini|  7800|     F|
|  5|Roshita|  7400|     F|
|  6|Shalini|  5800|     F|
+---+-------+------+------+



In [97]:
emp_df_male = emp_df.filter("gender=='M' and salary >6000")
emp_df_male.show()

+---+-----+------+------+
| id| name|salary|gender|
+---+-----+------+------+
|  2|Rohan|  7800|     M|
|  3|Rohit|  9800|     M|
+---+-----+------+------+



In [98]:
emp_df_female=emp_df.filter("gender=='F' and id > 5")
emp_df_female.show()

+---+-------+------+------+
| id|   name|salary|gender|
+---+-------+------+------+
|  6|Shalini|  5800|     F|
+---+-------+------+------+



In [99]:
max_sal_df=emp_df.agg({"salary":"max"})
max_sal_df.show()

+-----------+
|max(salary)|
+-----------+
|       9800|
+-----------+



In [100]:
max_sal_gender = emp_df.groupBy("gender").agg({"salary":"max"})
max_sal_gender.show()

+------+-----------+
|gender|max(salary)|
+------+-----------+
|     M|       9800|
|     F|       7800|
+------+-----------+



In [101]:
avg_sal_gender = emp_df.groupBy("gender").\
                        agg({"salary":"avg"}).\
                        withColumnRenamed("avg(salary)","avg_sal")
# avg_sal_gender.show()
emp_df_sal = emp_df.join(avg_sal_gender,"gender","left")\
                   .where("salary>avg_sal")\
                   .select("id","name","salary")
# filter(f"gender={avg_sal_gender.gender}")
emp_df_sal.show()

+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  2|  Rohan|  7800|
|  3|  Rohit|  9800|
|  4| Rohini|  7800|
|  5|Roshita|  7400|
+---+-------+------+



In [102]:
avg_sal = emp_df.agg(avg("salary")).withColumnRenamed("avg(salary)","avg_sal")
avg_sal_emp=avg_sal.collect()[0][0]
print(avg_sal_emp)
sal_gt_avg=emp_df.filter(f"salary > {avg_sal_emp}")
sal_gt_avg.show()

7266.666666666667
+---+-------+------+------+
| id|   name|salary|gender|
+---+-------+------+------+
|  2|  Rohan|  7800|     M|
|  3|  Rohit|  9800|     M|
|  4| Rohini|  7800|     F|
|  5|Roshita|  7400|     F|
+---+-------+------+------+

